In [3]:
import os
import pandas as pd
from kaggle.api.kaggle_api_extended import KaggleApi
import sqlite3

# Load the uploaded data files
api_kaggle = KaggleApi()
api_kaggle.authenticate()

data_address = 'imtkaggleteam/co-and-greenhouse-gas-emissions'
data_address2 = 'subhamjain/temperature-of-all-countries-19952020'

api_kaggle.dataset_download_files(data_address, path='./data', unzip=True)
api_kaggle.dataset_download_files(data_address2, path='./data2', unzip=True)

directory = [x for x in os.listdir('./data') if x.endswith('.csv')]
directory2 = [x for x in os.listdir('./data2') if x.endswith('.csv')]
green = pd.read_csv(os.path.join('./data',directory[1]))
temperature = pd.read_csv(os.path.join('./data2',directory2[0]))

# Data cleaning
city_temperature = temperature
co2_emissions = green

city_temperature = city_temperature[city_temperature['AvgTemperature'] != -99]
city_temperature['Country'] = city_temperature['Country'].replace({'US': 'United States'})
city_temperature['AvgTemperature'] = (city_temperature['AvgTemperature'] - 32) * (5/9)

# Data transformation
annual_temp = city_temperature.groupby(['Year', 'Country']).agg({'AvgTemperature': 'mean'}).reset_index()
co2_emissions = co2_emissions.rename(columns={'Entity': 'Country', 'Annual CO₂ emissions': 'Annual CO2 emissions'})

display(annual_temp)
display(co2_emissions)

# Merging datasets based on year and country
merged_data = pd.merge(annual_temp, co2_emissions, on=['Year', 'Country'])
os.makedirs('data', exist_ok=True)

# Database Storage
path = "../data/merged_data.db"
conn = sqlite3.connect(path)
merged_data.to_sql('merged_data', conn, if_exists='replace', index=False)
conn.close()
display(merged_data)


# Checking the years coverage in both datasets
years_temp = annual_temp['Year'].unique()
years_co2 = co2_emissions['Year'].unique()
common_years = set(years_temp).intersection(set(years_co2))
print(f"Years covered in temperature data: {sorted(years_temp)}")
print(f"Years covered in CO2 emissions data: {sorted(years_co2)}")
print(f"Common years: {sorted(common_years)}")

# Checking the countries coverage in both datasets
countries_temp = annual_temp['Country'].unique()
countries_co2 = co2_emissions['Country'].unique()
common_countries = set(countries_temp).intersection(set(countries_co2))
print(f"Countries covered in temperature data: {sorted(countries_temp)}")
print(f"Countries covered in CO2 emissions data: {sorted(countries_co2)}")
print(f"Common countries: {sorted(common_countries)}")





Dataset URL: https://www.kaggle.com/datasets/imtkaggleteam/co-and-greenhouse-gas-emissions
Dataset URL: https://www.kaggle.com/datasets/subhamjain/temperature-of-all-countries-19952020


/var/folders/s3/b_yxy3553dj6d47s232rl4xc0000gp/T/ipykernel_1142/2428975915.py:19: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  temperature = pd.read_csv(os.path.join('./data2',directory2[0]))
/var/folders/s3/b_yxy3553dj6d47s232rl4xc0000gp/T/ipykernel_1142/2428975915.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_temperature['Country'] = city_temperature['Country'].replace({'US': 'United States'})
/var/folders/s3/b_yxy3553dj6d47s232rl4xc0000gp/T/ipykernel_1142/2428975915.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

,Year,Country,AvgTemperature
0,1995,Algeria,18.005816
1,1995,Argentina,16.500761
2,1995,Australia,16.389863
3,1995,Austria,10.139574
4,1995,Bahamas,25.061661
...,...,...,...
3059,2020,Uruguay,20.458955
3060,2020,Uzbekistan,10.771144
3061,2020,Venezuela,26.812030
3062,2020,Vietnam,21.528192


,Country,Code,Year,Annual CO2 emissions
0,Afghanistan,AFG,1949,14656.0
1,Afghanistan,AFG,1950,84272.0
2,Afghanistan,AFG,1951,91600.0
3,Afghanistan,AFG,1952,91600.0
4,Afghanistan,AFG,1953,106256.0
...,...,...,...,...
30303,Zimbabwe,ZWE,2018,10714598.0
30304,Zimbabwe,ZWE,2019,9775428.0
30305,Zimbabwe,ZWE,2020,7849639.0
30306,Zimbabwe,ZWE,2021,8396158.0


,Year,Country,AvgTemperature,Code,Annual CO2 emissions
0,1995,Algeria,18.005816,DZA,9.677692e+07
1,1995,Argentina,16.500761,ARG,1.252398e+08
2,1995,Australia,16.389863,AUS,3.050556e+08
3,1995,Austria,10.139574,AUT,6.404406e+07
4,1995,Bahamas,25.061661,BHS,1.839328e+06
...,...,...,...,...,...
2868,2020,United States,8.555377,USA,4.714628e+09
2869,2020,Uruguay,20.458955,URY,6.477168e+06
2870,2020,Uzbekistan,10.771144,UZB,1.096320e+08
2871,2020,Venezuela,26.812030,VEN,6.195624e+07


Years covered in temperature data: [1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
Years covered in CO2 emissions data: [1750, 1751, 1752, 1753, 1754, 1755, 1756, 1757, 1758, 1759, 1760, 1761, 1762, 1763, 1764, 1765, 1766, 1767, 1768, 1769, 1770, 1771, 1772, 1773, 1774, 1775, 1776, 1777, 1778, 1779, 1780, 1781, 1782, 1783, 1784, 1785, 1786, 1787, 1788, 1789, 1790, 1791, 1792, 1793, 1794, 1795, 1796, 1797, 1798, 1799, 1800, 1801, 1802, 1803, 1804, 1805, 1806, 1807, 1808, 1809, 1810, 1811, 1812, 1813, 1814, 1815, 1816, 1817, 1818, 1819, 1820, 1821, 1822, 1823, 1824, 1825, 1826, 1827, 1828, 1829, 1830, 1831, 1832, 1833, 1834, 1835, 1836, 1837, 1838, 1839, 1840, 1841, 1842, 1843, 1844, 1845, 1846, 1847, 1848, 1849, 1850, 1851, 1852, 1853, 1854, 1855, 1856, 1857, 1858, 1859, 1860, 1861, 1862, 1863, 1864, 1865, 1866, 1867, 1868, 1869, 1870, 1871, 1872, 1873, 1874, 1875, 1876, 1877, 18